Pandas는 대부분의 연산기능을 Numpy로 부터 상속 받았음(UFuncs 포함)

게다가 인덱스와 레이블을 보존하여 다른 데이터에서 가져온 데이터들의 결합을 용이하게 한다 

유니버셜 함수:인덱스 보존

In [11]:
import pandas as pd
import numpy as np

In [12]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0,10,4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [13]:
df = pd.DataFrame(rng.randint(0,10, (3,4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [14]:
np.exp(ser) # 인덱스 보존됨

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [15]:
np.sin(df* np.pi/4) # 인덱스 보존됨

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


유니버셜 함수: 인덱스 정렬 = 인덱스가 보전되기 때문에 장점이 많음 

Series에서 인덱스 정렬

In [16]:

area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                 'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                       'New York': 19651127}, name='population')

In [17]:
population/area # 두 배열의 인덱스의 합집합을 보여주고 계산이 불가한 것은 NaN으로 표현

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [18]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

In [19]:
A = pd.Series([2,4,6], index=[0,1,2])
B = pd.Series([1,3,5], index=[1,2,3])
A+B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [20]:
A.add(B)

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [21]:
A.add(B, fill_value=0) # fill_value를 활용하여 A혹은 B에 누락된 값을 채워서 계산해줌

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

DataFrame 에서 인덱스 정렬

In [22]:
A = pd.DataFrame(rng.randint(0, 20, (2,2)), columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [23]:
B = pd.DataFrame(rng.randint(0,10,(3,3)), columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [24]:
A+B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


Signature: A.stack(level=-1, dropna=True)
Docstring:
Pivot a level of the (possibly hierarchical) column labels, returning a
DataFrame (or Series in the case of an object with a single level of
column labels) having a hierarchical index with a new inner-most level
of row labels.
The level involved will automatically get sorted.

Parameters
----------
level : int, string, or list of these, default last level
    Level(s) to stack, can pass level name
dropna : boolean, default True
    Whether to drop rows in the resulting Frame/Series with no valid
    values

Examples
----------
>>> s
     a   b
one  1.  2.
two  3.  4.

>>> s.stack()
one a    1
    b    2
two a    3
    b    4

Returns
-------
stacked : DataFrame or Series
File:      c:\users\jsong\anaconda3\lib\site-packages\pandas\core\frame.py
Type:      method

In [25]:
# fill_value를 통해 NaN에 숫자를 채울 수 있음
fill = A.stack().mean()
fill

4.5

In [26]:
A.add(B, fill_value=fill) # 참조 p.135 - 파이선 연산자와 pandas 메서드 매핑

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


유니버셜 함수: DataFrame과 Series 간의 연산 = 2차원 Numpy와 1차원 numpy간의 연산과 유사

In [27]:
A = rng.randint(10, size=(3,4)) # 일반 연산의 예
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [28]:
A -A[0] # numpy의 브로드캐스팅 룰을 따름

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

In [29]:
# pandas 도 연산 규칙이 기본적으로 행 방향으로 적용된다.
df = pd.DataFrame(A, columns=list('QRST')) # list 함수가 적용된 것임
df

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


In [30]:
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


In [47]:
# 열방향으로 적용하고 싶다면 pandas 메서드를 쓰고 axis=columns 키워드 매개 변수를 지정한다

df.subtract(df['R'], axis='rows')

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


In [32]:
halfrows = df.iloc[0:2, :] # 연습
halfrows

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8


In [33]:
halfrow = df.iloc[0, ::2] # 행이 하나이고 columns만 존재하면 표현 방식이 바뀌는 것 같음???
halfrow

Q    3
S    2
Name: 0, dtype: int32

In [34]:
df - halfrow #인덱스와 열을 맞추고 보존하고 있음(즉, halfrow가 원래 df의 첫번째 행의 Q열 S열임을 유지함)

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN
